In [24]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [25]:
from grader import score

# NLP: Analyzing Review Text

Unstructured data makes up the vast majority of data.  This is a basic intro to handling unstructured data.  Our objective is to be able to extract the sentiment (positive or negative) from review text.  We will do this from Yelp review data.

The first three questions task you to build models, of increasing complexity, to predict the rating of a review from its text.  These models will be assessed based on the root mean squared error of the number of stars predicted.  There is a reference solution (which should not be too hard to beat) that defines the score of 1.

The final question asks only for the result of a calculation, and your results will be compared directly to those of a reference solution.

## A note on scoring
It **is** possible to score >1 on these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!

## Download and parse the data

To start, let's download the dataset from Amazon S3:

In [ ]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_review.json.gz'

The training data are a series of JSON objects, in a gzipped file. Python supports gzipped files natively: [gzip.open](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [ujson](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [101]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review.json.gz') as f:
    data = [json.loads(line) for line in f]

In [ ]:
import gzip
import ujson as json
import random

#Fraction to subsample, needs to be between 0 and 1
subsample = 0.1

with gzip.open('yelp_train_academic_dataset_review.json.gz') as f:
    data = [json.loads(line) for line in f if random.random() < subsample]

Scikit Learn will want the labels in a separate data structure, so let's pull those out now.

In [ ]:
stars = [row['stars'] for row in data]

In [30]:
print len(stars)
print data[:1]
print data[0]['text']

1012913
[{u'votes': {u'funny': 0, u'useful': 2, u'cool': 1}, u'user_id': u'Xqd0DzHaiyRqVH3WRG7hzg', u'review_id': u'15SdjuK7DmYqUAj6rjGowg', u'text': u"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.", u'business_id': u'vcNAWiLM4dR7D2nwwJ7nCA', u'stars': 5, u'date': u'2007-05-17', u'type': u'review'}]
dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospit

### Notes:
1. [Pandas](http://pandas.pydata.org/) is able to read JSON text directly.  Use the `read_json()` function with the `lines=True` keyword argument.  While the rest of this notebook will assume you are using a list of dictionaries, you can complete it with dataframes, if you so desire. Some of the example code will need to be modified in this case.

2. There are obvious miscodings in the data.  There is no need to try to correct them.

## Building models

For the first three questions, you will need to build and train an estimator to predict the star rating from the text of a review.  We recommend building a pipeline out of transformers and estimators provided by Scikit Learn.  You can decide whether these pipelines should take full review objects or just their text as input to the `fit()` and `predict()` methods, but it does pay to be consistent.

You may find it useful to serialize the trained models to disk.  This will allow you to reload the models after restarting the notebook, without needing to retrain them.  We recommend using the [`dill` library](https://pypi.python.org/pypi/dill) for this (although the [`joblib` library](http://scikit-learn.org/stable/modules/model_persistence.html) also works).  Use
```python
dill.dump(estimator, open('estimator.dill', 'w'))
```
to serialize the object `estimator` to the file `estimator.dill`.  If you have trouble with this, try setting the `recurse=True` keyword args in the call of `dill.dump()`.  The estimator can be deserialized with
```python
estimator = dill.load(open('estimator.dill', 'r'))
```

You may run into trouble with the size of your models and Digital Ocean's memory limit. This is a major concern in real-world applications. Your production environment will likely not be that different from Digital Ocean and being able to deploy there is important. Think about what information the different stages of
your pipeline need and how you can reduce the memory footprint.

Additionally, you may notice that your serialized models are very large and take a long time to load.  Some hints to reduce their size:

- If you are using `GridSearchCV` to find the optimal values of hyperparameters (and you should be), the resultant object will contain many copies of the estimator that aren't needed any more.  Instead of serializing the whole `GridSearchCV`, serialize just the estimator with the correct hyperparameters.  This can be accessed through the `.best_estimator_` attribute of the `GridSearchCV` object.  Alternatively, the `.best_params_` attribute gives the best values of the hyperparameters.

- The `CountVectorizer` keeps track of all words that were excluded from vectorization in its `.stop_words_` attribute.  This can be interesting to examine, but isn't needed for predictions.  Set this attribute to the empty list before serializing it to save disk space.

In [3]:
import dill
import numpy as np
from collections import defaultdict


In [4]:
from sklearn import base
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.feature_extraction import DictVectorizer
from sklearn import datasets, linear_model, utils, preprocessing
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn import neighbors
from sklearn import ensemble
from sklearn.pipeline import Pipeline, FeatureUnion

# Questions

Each of the "model" questions asks you to create a function that models the number of stars given in a review from the review text.  It will be passed a list of dictionaries.  Each of these will have the same format as the JSON objects you've just read in.  This function should return a list of numbers of the same length, giving the predicted star ratings.

This function is passed to the `score()` function, which will receive input from the grader, run your function with that input, report the results back to the grader, and print out the score the grader returned.  Depending on how you constructed your estimator, you may be able to pass the predict method directly to the `score()` function.  If not, you will need to write a small wrapper function to mediate the data types.

## 1. bag_of_words_model
Build a linear model predicting the star rating based on the count of the words in each document (bag-of-words model).  Use a [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) or [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) to produce a feature matrix giving the counts of each word in each review.  Feed this in to linear model, such as `Ridge` or `SGDRegressor`, to predict the number of stars from each review.

**Hints**:
1. Don't forget to use tokenization!  This is important for good performance but it is also the most expensive step.  Try vectorizing as a first initial step and then running grid-serach and cross-validation only on of this pre-processed data.  `CountVectorizer` has to memorize the mapping between words and the index to which it is assigned.  This is linear in the size of the vocabulary.  The `HashingVectorizer` does not have to remember this mapping and will lead to much smaller models.

 ```python
 from sklearn.feature_extraction.text import CountVectorizer

 text = [row['text'] for row in data]
 X = CountVectorizer().fit_transform(text)

 # Now, this can be run with many different parameters
 # without needing to retrain the vectorizer:
 model.fit(X, stars, hyperparameter=something)
 ```

2. Try choosing different values for `min_df` (minimum document frequency cutoff) and `max_df` in `CountVectorizer`.  Setting `min_df` to zero admits rare words which might only appear once in the entire corpus.  This is both prone to overfitting and makes your data unmanageably large.  Don't forget to use cross-validation or to select the right value.  Notice that `HashingVectorizer` doesn't support `min_df`  and `max_df`.  However, it's not hard to roll your own transformer that solves for these.

3. Try using [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) or [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV).  If the memory footprint is too big, try switching to [Stochastic Gradient Descent](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) You might find that even ordinary linear regression fails due to the data size.  Don't forget to use [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to determine the regularization parameter!  How do the regularization parameter `alpha` and the values of `min_df` and `max_df` from `CountVectorizer` change the answer?

4. You will likely pick up several hyperparameters between the tokenization step and the regularization of the estimator.  While is is more strictly correct to do a grid search over all of them at once, this can take a long time. Quite often, doing a grid search over a single hyperparameter at a time can produce similar results.  Alternatively, the grid search may be done over a smaller subset of the data, as long as it is representative of the whole.

5. Finally, assemble a pipeline that will transform the data from records all the way to predictions.  This will allow you to submit its predict method to the grader for scoring.

In [5]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    # modified only one column allowed
    def __init__(self, col_name):
        self.col_name = col_name  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        data = []
        col = self.col_name
        for row in X:
            new_row = row[col]
            data.append(new_row)
        return data


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#text = [row['text'] for row in data]
text = [ data[0]['text'], data[1]['text']]
print text

X = CountVectorizer().fit_transform(text)
X

In [ ]:
mydata = data[:20000]
mystars = stars[:20000]

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

bag_of_words_est = Pipeline([
    ('trans', ColumnSelectTransformer('text')),# Column selector (remember the ML project?)
    ('cvec', CountVectorizer(min_df=1, max_df=1.0, ngram_range=(1,2))),# Vectorizer
    # Frequency filter (if necessary)
    ('ridgereg', Ridge(alpha=95))# Regressor
])

bag_of_words_est.fit(mydata, mystars)

In [ ]:
frang = [90+p/10.0 for p in range(0, 10)]
print frang

In [ ]:
from sklearn.model_selection import GridSearchCV

frang = [90+p/10.0 for p in range(0, 10)]

param_grid = {'cvec__min_df': xrange(1,2,1) ,
              'ridgereg__alpha': xrange(900,1200,100),
              #'cvec__max_df': [0.8,0.9,0.95,1.0] ,
             }

grid_search = GridSearchCV(bag_of_words_est, param_grid=param_grid, verbose=10)
grid_search.fit(mydata, mystars) 

In [ ]:
print(grid_search.best_estimator_.get_params())
best_q1 = grid_search.best_estimator_

In [ ]:
score('nlp__bag_of_words_model', best_q1.predict)  #bag_of_words_est

## 2. normalized_model
Normalization is key for good linear regression. Previously, we used the count as the normalization scheme.  Add in a normalization transformer to your pipeline to improve the score.  Try some of these:

1. You can use the "does this word present in this document" as a normalization scheme, which means the values are always 1 or 0.  So we give no additional weight to the presence of the word multiple times.

2. Try using the log of the number of counts (or more precisely, $log(x+1)$). This is often used because we want the repeated presence of a word to count for more but not have that effect tapper off.

3. [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a common normalization scheme used in text processing.  Use the [TFIDFTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer). There are options for using `idf` and taking the logarithm of `tf`.  Do these significantly affect the result?

Finally, if you can't decide which one is better, don't forget that you can combine models with a linear regression.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [28]:
normalized_est = Pipeline([
    ('trans', ColumnSelectTransformer('text')),# Column selector (remember the ML project?)
    ('cvec', CountVectorizer(min_df=1, max_df=1.0, ngram_range=(1,2))),# Vectorizer
    ('tfvec', TfidfTransformer()),# Vectorizer
    # Frequency filter (if necessary)
    ('ridgereg', Ridge(alpha=1))# Regressor
])


In [24]:
param_grid = {'ridgereg__alpha': xrange(2,20,4),
              #'cvec__min_df': xrange(1,2,1) , 
              #'cvec__max_df': [0.8,0.9,0.95,1.0] ,
             }

grid_search = GridSearchCV( normalized_est, param_grid=param_grid, verbose=10)
grid_search.fit(mydata, mystars) 

[CV] ............... ridgereg__alpha=10, score=0.371539, total=  12.1s
[CV] ridgereg__alpha=10 ..............................................


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.4min remaining:    0.0s


[CV] ............... ridgereg__alpha=10, score=0.403140, total=  12.7s
[CV] ridgereg__alpha=14 ..............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.7min remaining:    0.0s


[CV] ............... ridgereg__alpha=14, score=0.340917, total=  12.2s
[CV] ridgereg__alpha=14 ..............................................
[CV] ............... ridgereg__alpha=14, score=0.330828, total=  12.3s
[CV] ridgereg__alpha=14 ..............................................
[CV] ............... ridgereg__alpha=14, score=0.360037, total=  11.8s
[CV] ridgereg__alpha=18 ..............................................
[CV] ............... ridgereg__alpha=18, score=0.304644, total=  11.8s
[CV] ridgereg__alpha=18 ..............................................
[CV] ............... ridgereg__alpha=18, score=0.298191, total=  12.1s
[CV] ridgereg__alpha=18 ..............................................
[CV] ............... ridgereg__alpha=18, score=0.326207, total=  12.2s


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  4.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('trans', ColumnSelectTransformer(col_name='text')), ('cvec', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'ridgereg__alpha': xrange(2, 22, 4)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [26]:
#normalized_est.fit(mydata, mystars) 
print(grid_search.best_estimator_.get_params())
best_q2 = grid_search.best_estimator_

{'cvec__analyzer': u'word', 'cvec__tokenizer': None, 'cvec__vocabulary': None, 'cvec__decode_error': u'strict', 'cvec': CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None), 'ridgereg__max_iter': None, 'tfvec__smooth_idf': True, 'ridgereg__fit_intercept': True, 'ridgereg__alpha': 2, 'ridgereg__tol': 0.001, 'cvec__lowercase': True, 'cvec__binary': False, 'tfvec__sublinear_tf': False, 'cvec__min_df': 1, 'cvec__encoding': u'utf-8', 'cvec__ngram_range': (1, 2), 'ridgereg__random_state': None, 'cvec__dtype': <type 'numpy.int64'>, 'cvec__token_pattern': u'(?u)\\b\\w\\w+\\b', 'tfvec__norm': u'l2', 'cvec__input': u'content', 'cvec__max_df': 1.0, 'cvec__max_features': None, 

In [27]:
score('nlp__normalized_model', best_q2.predict) #normalized_est

Your score:  1.19426940833


## 3. bigram_model
In a bigram model, we'll consider both single words and pairs of consecutive words that appear.  This is going to be a much higher dimensional problem (large $p$) so you should be careful about overfitting.

Sometimes, reducing the dimension can be useful.  Because we are dealing with a sparse matrix, we have to use [TruncatedSVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD).  If we reduce the dimensions, we can use a more sophisticated models than linear ones.

As before, memory problems can crop up due to the engineering constraints. Playing with the number of features, using the `HashingVectorizer`, incorporating `min_df` and `max_df` limits, and handling stop-words in some way are all methods of addressing this issue. If you are using `CountVectorizer`, it is possible to run it with a fixed vocabulary (based on a training run, for instance). Check the documentation.

**A side note on multi-stage model evaluation:** When your model consists of a pipeline with several stages, it can be worthwhile to evaluate which parts of the pipeline have the greatest impact on the overall accuracy (or other metric) of the model. This allows you to focus your efforts on improving the important algorithms, and leaving the rest "good enough".

One way to accomplish this is through ceiling analysis, which can be useful when you have a training set with ground truth values at each stage. Let's say you're training a model to extract image captions from websites and return a list of names that were in the caption. Your overall accuracy at some point reaches 70%. You can try manually giving the model what you know are the correct image captions from the training set, and see how the accuracy improves (maybe up to 75%). Alternatively, giving the model the perfect name parsing for each caption increases accuracy to 90%. This indicates that the name parsing is a much more promising target for further work, and the caption extraction is a relatively smaller factor in the overall performance.

If you don't know the right answers at different stages of the pipeline, you can still evaluate how important different parts of the model are to its performance by changing or removing certain steps while keeping everything else constant. You might try this kind of analysis to determine how important adding stopwords and stemming to your NLP model actually is, and how that importance changes with parameters like the number of features.

In [29]:
score('nlp__bigram_model', best_q2.predict) #bigram_est

Your score:  1.19495328603


## 4. food_bigrams
Look over all reviews of restaurants.  You can determine which businesses are restaurants by looking in the `yelp_train_academic_dataset_business.json.gz` file from the ml project or downloaded below.

In [33]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

download: s3://dataincubator-course/mldata/yelp_train_academic_dataset_business.json.gz to ./yelp_train_academic_dataset_business.json.gz


In [96]:
import gzip
import ujson as json
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    business_data = [json.loads(line) for line in f]

In [97]:
business_data[1]['categories']

[u'Restaurants']

Each row of this file corresponds to a single business.  The category key gives a list of categories for each; take all where "Restaurants" appears.

In [98]:
restaurant_ids = [a for a in business_data if 'Restaurants' in a['categories']]

In [99]:
assert len(restaurant_ids) == 12876

In [102]:
bid = restaurant_ids[3]['business_id']
bidrev = data[3]['business_id']
print bid
print bidrev
data[3]['review_id']
bids = [ bid['business_id'] for bid in restaurant_ids ]
bids_set = set(bids)
print len(bids)
print len(bids_set)

RgDg-k9S5YD_BaxMckifkg
vcNAWiLM4dR7D2nwwJ7nCA
12876
12876


In [103]:
restaurant_reviews = [ d['text'] for d in data if d['business_id'] in bids_set]

### The "business_id" here is the same as in the review data.  Use this to extract the review text for all reviews of restaurants.

In [3]:
print restaurant_reviews[20:50]

[u'Very nice and clean place to have breakfast or lunch', u'I eat here regularily because it is consistently good.  Very large menu which makes it tough to choose but never disappointed!', u"Unlimited hot coffee. I don't have any recollection of what I ate....\n\nI wanted t give it four stars, but that would be a very generous review for a meal I cannot recall.\n\nI know it was decently good, and it's like the only restaurant in Deforest.\n\nFor that, I give it four stars.", u"Homestyle cooking at its best, dnt go to culvers, McDonald's or whatever trash is right off the hwy, head a bit further down u will find this gem, one of the reasons I love this place so much? They dnt play annoying music, scratch that, they don't play ANY music. People working here are super nice as to be expected. If I owned this joint, I would invest in a new bigger sign, as I drove right by this place and almost missed it twice.", u'Nice simple homey diner. Very friendly staff, huge family friendly menu, sala

In [2]:
assert len(restaurant_reviews) == 574278

In [78]:
import dill
dill.dump(restaurant_reviews, open('restaurant_reviews.dill', 'w'))

In [1]:
import dill
restaurant_reviews = dill.load(open('restaurant_reviews.dill', 'r'))

We want to find collocations --- that is, bigrams that are "special" and appear more often than you'd expect from chance. We can think of the corpus as defining an empirical distribution over all ngrams.  We can find word pairs that are unlikely to occur consecutively based on the underlying probability of their words. Mathematically, if $p(w)$ be the probability of a word $w$ and $p(w_1 w_2)$ is the probability of the bigram $w_1 w_2$, then we want to look at word pairs $w_1 w_2$ where the statistic

  $$ \frac{p(w_1 w_2)}{p(w_1) p(w_2)} $$

is high.  ** Return the top 100 (mostly food) bigrams with this statistic with the 'right' prior factor** (see below).

 Estimating the probabilities is simply a matter of counting, and there are number of approaches that will work.  
 One is to use one of the tokenizers to count up how many times each word and each bigram appears in each review, and then sum those up over all reviews.  
#### You might want to know that the `CountVectorizer` has a `.get_feature_names()` method which gives the string associated with each column.  

(Question for thought: Why doesn't the `HashingVectorizer` have a similar method?)

 *Questions:* This statistic is a ratio and problematic when the denominator is small.  We can fix this by applying  Bayesian smoothing to $p(w)$ (i.e. mixing the empirical distribution with the uniform distribution over the vocabulary).

 1. How does changing this smoothing parameter affect the word pairs you get qualitatively?

 2. We can interpret the smoothing parameter as adding a constant number of occurrences of each word to our distribution.  Does this help you determine set a reasonable value for this 'prior factor'?

 3. For fun: also check out [Amazon's Statistically Improbable Phrases](http://en.wikipedia.org/wiki/Statistically_Improbable_Phrases).

 *Implementation note:*
 As you adjust the size of the Bayesian smoothing parameter, you will notice first nonsense phrases being removed and then legitimate bigrams being removed, leaving you with only generic bigrams.  The goal is to find a value of the smoothing parameter between these two transitions.

 The reference solution is not an aggressive filterer: it errors in favor of leaving apparently nonsensical words. On further consideration, many of these are actually somewhat meaningful. The smoothing parameter chosen in the reference solution is equivalent to giving each word ** 90**  previous appearances prior to considering this data.  This was chosen by generating a list of bigrams for a range of smoothing parameters and seeing how many of the bigrams were shared between neighboring values.  When the shared fraction reached 95%, we judged the solution to have converged.  Note that `min_df` should not be set too high, where it could exclude these borderline words.

In [7]:
assert len(restaurant_reviews) == 574278

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
counter_single = CountVectorizer(ngram_range=(1,1))

counts1 = counter_single.fit_transform(restaurant_reviews)

In [27]:
counter_double = CountVectorizer(analyzer='word',ngram_range=(2,2),min_df=20)

counts2 = counter_double.fit_transform(restaurant_reviews)

In [28]:
print counts1.shape
print counts2.shape
#print counts.toarray() 
#print counts1.toarray()#dense matrix
print counts2[0]
print type(counts2)

(574278, 173736)
(574278, 259564)
  (0, 120414)	1
  (0, 74826)	1
  (0, 62518)	1
  (0, 245180)	1
  (0, 249228)	1
  (0, 246115)	1
  (0, 218191)	1
  (0, 150454)	1
  (0, 100121)	1
  (0, 48909)	1
  (0, 124987)	1
  (0, 245348)	1
  (0, 169585)	1
  (0, 179034)	1
  (0, 216573)	1
  (0, 21600)	1
  (0, 59926)	1
  (0, 232173)	1
<class 'scipy.sparse.csr.csr_matrix'>


In [9]:
sum_count1 = counts1.sum(axis=0)
print sum_count1.shape

(1, 173736)


In [29]:
sum_count2 = counts2.sum(axis=0)
print sum_count2.shape

(1, 259564)


In [30]:
print sum_count1
cut =0
ctlist1 = sum_count1.tolist()[0]
ctlist2 = sum_count2.tolist()[0]
#for c in sum_count1.tolist()[0]:
#    print c
#    cut += 1
#    if cut >10:
#        break
print len(ctlist1)
print len(ctlist2)


[[11129   623     1 ...,     1     1     1]]
173736
259564


In [31]:
names1 = counter_single.get_feature_names()

In [32]:
names2 = counter_double.get_feature_names()

In [33]:
print names1[1100:1105], names2[1100:1105]
print len(names1)
print len(names2)

[u'1666666', u'1668', u'167', u'1677', u'168'] [u'21 to', u'21 we', u'21 year', u'21 years', u'215 and']
173736
259564


In [34]:
dic1 = dict(zip(names1, ctlist1))

In [35]:
dic1.items()[0]

(u'fawn', 22L)

In [36]:
dic2 = dict(zip(names2, ctlist2))

In [37]:
dic2p = dic2

In [38]:
for k,v in dic2.items():
    if v>9000:
        print k,v, k.split(' ')
    w = k.split(' ')
    dic2p[k] = 1.0*v/(dic1[w[0]]+90.0)/(dic1[w[1]]+90.0)


they don 10281 [u'they', u'don']
to find 19466 [u'to', u'find']
customer service 12061 [u'customer', u'service']
of their 21214 [u'of', u'their']
delicious the 9531 [u'delicious', u'the']
restaurant is 12360 [u'restaurant', u'is']
the beef 10174 [u'the', u'beef']
out the 18009 [u'out', u'the']
and we 40377 [u'and', u'we']
on their 10579 [u'on', u'their']
at this 15138 [u'at', u'this']
and not 24120 [u'and', u'not']
to have 30729 [u'to', u'have']
the menu 60410 [u'the', u'menu']
was pretty 26432 [u'was', u'pretty']
salad and 11274 [u'salad', u'and']
were very 15429 [u'were', u'very']
to order 24760 [u'to', u'order']
the burger 11203 [u'the', u'burger']
our waitress 10353 [u'our', u'waitress']
this time 10656 [u'this', u'time']
worth the 16540 [u'worth', u'the']
but we 15448 [u'but', u'we']
which was 42884 [u'which', u'was']
ve ever 17441 [u've', u'ever']
here is 13715 [u'here', u'is']
is just 12840 [u'is', u'just']
the restaurant 44131 [u'the', u'restaurant']
you have 24793 [u'you', u'h

In [39]:
from collections import OrderedDict

sorted_dic = OrderedDict(sorted(dic2p.items(), key=lambda t: -t[1]))

In [40]:
c = 0
q4 = []
for a in sorted_dic:
    print a, sorted_dic[a]
    q4.append(a)
    c+=1
    if c==100:
        break

ezzyujdouig4p gyb3pv_a 0.00277744233788
hodge podge 0.00258620689655
himal chuli 0.00251607492312
hoity toity 0.00251198903859
roka akor 0.00245098039216
f_5_unx wrafcxuakbzrdw 0.00245043439519
knick knacks 0.00243463533397
reina pepiada 0.00242052023121
cien agaves 0.00240286990659
itty bitty 0.00236317627622
khai hoan 0.00234192037471
riff raff 0.00231061231226
baskin robbins 0.00228190932216
grana padano 0.00225076855512
tutti santi 0.00224285268149
ropa vieja 0.00219705659036
gulab jamun 0.00218431454129
pel meni 0.00216346153846
ore ida 0.00216086434574
laan xang 0.00213971778857
dac biet 0.00213430408752
rula bula 0.0021297353893
hu tieu 0.00209930631617
bandeja paisa 0.00207100591716
tammie coe 0.00206388206388
chicha morada 0.00204600752046
alain ducasse 0.00202571094663
dol sot 0.00200534759358
itsy bitsy 0.00199724517906
mille feuille 0.00198412698413
feng shui 0.00198075834748
hors oeuvres 0.00194298455168
marche bacchus 0.00191161866931
uuu uuu 0.00191112411976
celine dion 

In [41]:
q4

[u'ezzyujdouig4p gyb3pv_a',
 u'hodge podge',
 u'himal chuli',
 u'hoity toity',
 u'roka akor',
 u'f_5_unx wrafcxuakbzrdw',
 u'knick knacks',
 u'reina pepiada',
 u'cien agaves',
 u'itty bitty',
 u'khai hoan',
 u'riff raff',
 u'baskin robbins',
 u'grana padano',
 u'tutti santi',
 u'ropa vieja',
 u'gulab jamun',
 u'pel meni',
 u'ore ida',
 u'laan xang',
 u'dac biet',
 u'rula bula',
 u'hu tieu',
 u'bandeja paisa',
 u'tammie coe',
 u'chicha morada',
 u'alain ducasse',
 u'dol sot',
 u'itsy bitsy',
 u'mille feuille',
 u'feng shui',
 u'hors oeuvres',
 u'marche bacchus',
 u'uuu uuu',
 u'celine dion',
 u'nanay gloria',
 u'doon varna',
 u'luc lac',
 u'krispy kreme',
 u'woonam jung',
 u'perrier jouet',
 u'deja vu',
 u'molecular gastronomy',
 u'puerto rican',
 u'vice versa',
 u'patatas bravas',
 u'sais quoi',
 u'cullen skink',
 u'lloyd wright',
 u'holyrood 9a',
 u'pura vida',
 u'lomo saltado',
 u'nuoc mam',
 u'wal mart',
 u'dueling pianos',
 u'valle luna',
 u'bradley ogden',
 u'rustler rooste',
 u'a

In [42]:
score('nlp__food_bigrams', lambda: q4)#["kare kare"] * 100)

Your score:  0.98


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*